# ODD Data Processing for Any Department

This notebook processes ODD (Órgano Deliberativo Departamental) data for a specified department in Uruguay. It performs the following tasks:

1. Data Loading: Reads Excel files containing ODD data from a specified folder.
2. Data Filtering: Extracts data for the chosen department.
3. Data Cleaning: Renames columns and cleans party names.
4. Zone Mapping: Adds zone information based on the series codes.
5. Data Saving: Saves the processed data at various stages of the pipeline.

The notebook is designed to be flexible, allowing users to process data for different departments by changing a few parameters.

Key Features:
- Handles multiple Excel files in the input folder
- Applies consistent data cleaning across all entries
- Maps series codes to geographical zones using predefined mappings
- Saves intermediate and final results for further analysis or verification

To use this notebook, you'll need to specify:
1. The department name
2. The input folder containing ODD Excel files
3. The output directory for processed data
4. The directory containing series-to-zone mapping files
5. The final directory for the fully processed data

The notebook will then process the data and save the results in the specified locations, providing a clear audit trail of the data processing steps.

In [3]:
import pandas as pd
import os
import json

def load_odd_data(odd_folder):
    all_data = []
    for file_name in os.listdir(odd_folder):
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(odd_folder, file_name)
            try:
                df = pd.read_excel(file_path, header=8, usecols='A:H')
                all_data.append(df)
            except Exception as e:
                print(f"Error reading file {file_name}: {e}")
    return pd.concat(all_data, ignore_index=True) if all_data else None

def clean_party_name(value):
    party = value.split('_-_')[0]
    party = party.replace('_', ' ')
    return party.replace(' O.D.D.', '')

def get_zone(serie, series_to_zona):
    series = str(serie).split()
    zones = [series_to_zona.get(s, 'Unknown') for s in series]
    return ' '.join(set(zones))

def process_odd_data(department, odd_folder, output_dir, mapping_dir, final_dir):
    # Load data
    combined_data = load_odd_data(odd_folder)
    if combined_data is None:
        print("No valid data found in Excel files.")
        return

    # Filter data for the specified department
    department_odd_data = combined_data[combined_data['DEPTO'] == department].copy()
    department_odd_data.to_csv(os.path.join(output_dir, f'{department.lower()}_odd_filtered.csv'), index=False, encoding='utf-8')

    # Clean and transform data
    department_odd_data.rename(columns={'CONVOCATORIA': 'PARTIDO'}, inplace=True)
    department_odd_data['PARTIDO'] = department_odd_data['PARTIDO'].apply(clean_party_name)
    department_odd_data.to_csv(os.path.join(output_dir, f'{department.lower()}_odd_cleaned.csv'), index=False, encoding='utf-8')

    if 'ACTO' in department_odd_data.columns:
        department_odd_data = department_odd_data.drop(columns=['ACTO'])
        department_odd_data.to_csv(os.path.join(output_dir, f'{department.lower()}_odd_no_acto.csv'), index=False, encoding='utf-8')

    # Add zone information
    mapping_file = os.path.join(mapping_dir, f'mapping_{department.lower()}.json')
    with open(mapping_file, 'r', encoding='utf-8') as f:
        series_to_zona = json.load(f)
    department_odd_data['ZONA'] = department_odd_data['SERIES'].apply(lambda x: get_zone(x, series_to_zona))
    
    # Save final data
    department_odd_data.to_csv(os.path.join(final_dir, f'{department.lower()}_odd_final.csv'), index=False, encoding='utf-8')

    print(f"Processed ODD data for {department}:")
    print(department_odd_data.head())
    print(f"\nSaved processed data to:")
    print(f"- {output_dir}/{department.lower()}_odd_filtered.csv")
    print(f"- {output_dir}/{department.lower()}_odd_cleaned.csv")
    if 'ACTO' in combined_data.columns:
        print(f"- {output_dir}/{department.lower()}_odd_no_acto.csv")
    print(f"- {final_dir}/{department.lower()}_odd_final.csv")

## Process ODD Data for a Specific Department

You can change the department name, input folder, and output directory in the cell below to process data for different departments.

In [4]:
import os

# Get the project root directory
project_root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

department = "Canelones"  # Change this to process different departments
odd_folder = os.path.join(project_root, "data", "raw", "results", "ODD")
output_dir = os.path.join(project_root, "data", "processed", "ODD", department)
mapping_dir = os.path.join(project_root, "maps", "zonasxseries")
final_dir = os.path.join(project_root, "data", "final", "odd", department)

# Ensure directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(final_dir, exist_ok=True)

process_odd_data(department, odd_folder, output_dir, mapping_dir, final_dir)

C:\Users\trico\AppData\Local\Temp\ipykernel_37436\667881016.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_data, ignore_index=True) if all_data else None


Processed ODD data for Canelones:
              PARTIDO      DEPTO CIRCUITO SERIES     ESCRUTINIO   HOJA  \
761  Asamblea Popular  Canelones        4    CAA  Departamental   1326   
762  Asamblea Popular  Canelones        5    CAA  Departamental  11917   
763  Asamblea Popular  Canelones        7    CAA  Departamental   1326   
764  Asamblea Popular  Canelones        8    CAA  Departamental  11917   
765  Asamblea Popular  Canelones       11    CAA  Departamental   1326   

    CNT_VOTOS       ZONA  
761         1  Canelones  
762         1  Canelones  
763         1  Canelones  
764         1  Canelones  
765         1  Canelones  

Saved processed data to:
- c:\Users\trico\Desktop\Internas 2024\data\processed\ODD\Canelones/canelones_odd_filtered.csv
- c:\Users\trico\Desktop\Internas 2024\data\processed\ODD\Canelones/canelones_odd_cleaned.csv
- c:\Users\trico\Desktop\Internas 2024\data\processed\ODD\Canelones/canelones_odd_no_acto.csv
- c:\Users\trico\Desktop\Internas 2024\data\final\